In [38]:
import glob
import os
import matplotlib as mlib
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt
from numpy import loadtxt
import re
import csv
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.patches as mpatches

In [39]:
#constants


In [40]:
# d0 for lectrons and muons separately\n
def impact_d0(param_point):
    
    folder='hepmc_'+param_point
    outputfolder='hepmc_output_'+param_point
    
    #reading pythia output
    files = sorted(glob.glob( folder+'/*'))
    print(files)

    
    os.makedirs(os.path.dirname(outputfolder+"/ctau_plus.dat"), exist_ok=True)
    open(outputfolder+"/d0_e.dat","w")
    open(outputfolder+"/d0_mu.dat","w")
    open(outputfolder+"/pT_e.dat","w")
    open(outputfolder+"/pT_mu.dat","w")
    #open hepmc file
    for filenum, file in enumerate(files):
        hepmc=[]
        d0_l=[]
        d0_e=[]
        d0_mu=[]
        pT_e=[]
        pT_mu=[]
        print(file)
        with open (file, 'r') as f:
            for rownum,row in enumerate(csv.reader(f,delimiter=' ')):
                if (len(row)>4):
                    hepmc.append(row)

        print(len(hepmc)) 
        
        counte=0
        countmu=0
        countevents=0
        countV0=0
        for rownum ,row in enumerate(hepmc):
            #print(hepmc[rownum][0])
            #finding vertex
            if (row[0]=='E'):
                countevents+=1
            if (row[0]=='V'):
                #print('V=',row)
                ip=rownum+1
                #finding electron/muon related to this vertex (knowing that max 3 particles can originate from it)
                for i in range(0,2): 
                    #print(hepmc[ip+i])
                    if (ip+i<len(hepmc)):
                        if (hepmc[ip+i][2]=='11'):
                            counte+=1
                            #print(hepmc[ip+i][2:6])
                            V=np.array(row[3:6], float)
                            P=np.array(hepmc[ip+i][3:6], float)

                            #calculating |d0|
                            VT=np.sqrt(V[0]**2+V[1]**2)
                            PT=np.sqrt(P[0]**2+P[1]**2)
                            
                            if ((V[0]!=0)&(V[1]!=0)&(V[2]!=0)): 
                                cTheta=(V[0]*P[0]+V[1]*P[1])/(VT*PT)
                                
                                d0_e.append(VT*np.sqrt(1-cTheta**2))
                                
                            else: 
                                countV0+=1
                                d0_e.append(0)
                                
                            pT_e.append(PT)    
                            
                            
                        if (hepmc[ip+i][2]=='13'):
                            countmu+=1
                            #print(hepmc[ip+i][2:6])
                            V=np.array(row[3:6], float)
                            P=np.array(hepmc[ip+i][3:6], float)
                            #print(P)

                            #calculating |d0|
                            VT=np.sqrt(V[0]**2+V[1]**2)
                            PT=np.sqrt(P[0]**2+P[1]**2)
                            
                            if ((V[0]!=0)&(V[1]!=0)&(V[2]!=0)):  
                                cTheta=(V[0]*P[0]+V[1]*P[1])/(VT*PT)
                                
                                d0_mu.append(VT*np.sqrt(1-cTheta**2))
                                
                            else: 
                                countV0+=1
                                d0_mu.append(0)
                            
                            pT_mu.append(PT)

                        i+=1
        #print(d0_l)
    
        print("# events = {}".format(countevents)+"# e = {}".format(counte)+"# mu = {}".format(countmu)\
              +"# V(primary) = {}".format(countV0))
        
        #Saving d0 to file
        with open(outputfolder+"/d0_e.dat", 'a+') as foute:
                        #fout.write('# d0 [mm]' + '\n')
                        np.savetxt(foute, d0_e, fmt='%7f')   
                    
        with open(outputfolder+"/d0_mu.dat", 'a+') as foutmu:
                        #fout.write('# d0 [mm]' + '\n')
                        np.savetxt(foutmu, d0_mu, fmt='%7f')
                    
        #Saving pT to file
        with open(outputfolder+"/pT_e.dat", 'a+') as foute:
                        np.savetxt(foute, pT_e, fmt='%7f')   
                    
        with open(outputfolder+"/pT_mu.dat", 'a+') as foutmu:
                        np.savetxt(foutmu, pT_mu, fmt='%7f')

In [63]:
# lifetime of chi_+
def lifetime(param_point):
    
    folder='hepmc_'+param_point
    outputfolder='hepmc_output_'+param_point
    print(folder)
    
    #reading pythia output
    files = sorted(glob.glob( folder+'/*'))
    print(files)

    os.makedirs(os.path.dirname(outputfolder+"/"+param_point+"_ctau_plus.dat"), exist_ok=True)
    open(outputfolder+"/"+param_point+"_ctau_plus.dat","w")
    open(outputfolder+"/"+param_point+"_DM.dat","w")
    open(outputfolder+"/"+param_point+"_beta_plus.dat","w")
    open(outputfolder+"/"+param_point+"_gamma_plus.dat","w")
    open(outputfolder+"/"+param_point+"_betagamma_plus.dat","w")
    open(outputfolder+"/"+param_point+"_pT_plus.dat","w")
    
    #open hepmc file
    for filenum, file in enumerate(files):
        hepmc=[]
        ctau_plus=[]
        beta_plus=[]
        gamma_plus=[]
        betagamma_plus=[]
        pT_plus=[]
        DM=[]

        print(file)
        with open (file, 'r') as f:
            for rownum,row in enumerate(csv.reader(f,delimiter=' ')):
                if (len(row)>4):
                    hepmc.append(row)

        print(len(hepmc)) 

        countevents=0
        countV0=0
        for rownum ,row in enumerate(hepmc):
            #print(hepmc[rownum][0])
            #finding vertex
            if (row[0]=='E'):
                countevents+=1
            
            #calculating beta and gamma factors of chi_plus
            if (row[2]=='9000007') or (row[2]=='-9000007'):
                
                plus=np.array(row[3:7], float)
                #print(plus)
                
                beta=np.sqrt(plus[0]**2+plus[1]**2+plus[2]**2)/plus[3]
                beta_plus.append(beta)
                gamma_plus.append(1/np.sqrt(1-beta**2))
                betagamma_plus.append(beta/np.sqrt(1-beta**2))
                pT_plus.append(np.sqrt(plus[0]**2+plus[1]**2))
                
            
            if (row[0]=='V'):
                #print('V=',row)
                ip=rownum+1
                #finding DM related to this vertex (knowing that max 3 particles can originate from it)
                for i in range(0,2): 
                    #print(hepmc[ip+i])
                    if (ip+i<len(hepmc)):
                        if (hepmc[ip+i][2]=='9000005'):
                            DM.append(hepmc[ip+i])
                            #print(hepmc[ip+i][2:6])
                            V=np.array(row[3:6], float)
                            P=np.array(hepmc[ip+i][3:6], float)

                            #calculating lifetime and time of flight in lab frame
                            ctau_plus.append(np.sqrt(V[0]**2+V[1]**2+V[2]**2))
            
                              

                        i+=1
        #print(d0_l)
    
        #print("# events = {}".format(countevents)+"# e = {}".format(counte)+"# mu = {}".format(countmu)\
        #      +"# V(primary) = {}".format(countV0))
        
        #Saving to files 
        with open(outputfolder+"/"+param_point+"_ctau_plus.dat", 'a+') as fout:
                        np.savetxt(fout, ctau_plus, fmt='%7f') 
                    
        with open(outputfolder+"/"+param_point+"_beta_plus.dat", 'a+') as fout:
                        np.savetxt(fout, beta_plus, fmt='%7f') 
                    
        with open(outputfolder+"/"+param_point+"_gamma_plus.dat", 'a+') as fout:
                        np.savetxt(fout, gamma_plus, fmt='%7f') 
                
        with open(outputfolder+"/"+param_point+"_betagamma_plus.dat", 'a+') as fout:
                        np.savetxt(fout, betagamma_plus, fmt='%7f') 
                
        with open(outputfolder+"/"+param_point+"_pT_plus.dat", 'a+') as fout:
                        np.savetxt(fout, pT_plus, fmt='%7f') 
                
        #print(DM)            
        #with open(outputfolder+"/DM.dat", 'a+') as fout:
        #                fout.write(DM)

In [42]:
# lifetime of chi_+
def lifetimeAngle(folder,outputfolder,tanmin,tanmax):
    #reading pythia output
    files = sorted(glob.glob( folder+'/*'))
    print(files)

    os.makedirs(os.path.dirname(outputfolder+"/ctau_plus.dat"), exist_ok=True)
    open(outputfolder+"/ctau_plus.dat","w")
    #open hepmc file
    for filenum, file in enumerate(files):
        hepmc=[]
        ctau_plus=[]

        print(file)
        with open (file, 'r') as f:
            for rownum,row in enumerate(csv.reader(f,delimiter=' ')):
                if (len(row)>4):
                    hepmc.append(row)

        print(len(hepmc)) 
        
        counte=0
        countmu=0
        countevents=0
        countAngle=0
        for rownum ,row in enumerate(hepmc):
            #print(hepmc[rownum][0])
            #finding vertex
            if (row[0]=='E'):
                countevents+=1
            if (row[0]=='V'):
                #print('V=',row)
                ip=rownum+1
                #finding DM related to this vertex (knowing that max 3 particles can originate from it)
                for i in range(0,2): 
                    #print(hepmc[ip+i])
                    if (ip+i<len(hepmc)):
                        if (hepmc[ip+i][2]=='9000005'):
                            countDM+=1
                            #print(hepmc[ip+i][2:6])
                            V=np.array(row[3:6], float)
                            P=np.array(hepmc[ip+i][3:6], float)

                            #calculating ctau in the range of angles:
                            VT=np.sqrt(V[0]**2+V[1]**2)
                            tantheta=VT/np.abs(V[2])
                            
                            if ((tanmin<tantheta)&(tantheta<tanmax)):
                                countAngle+=1
                                ctau_plus.append(np.sqrt(V[0]**2+V[1]**2+V[2]**2))

                              

                        i+=1
        #print(d0_l)
    
        print("# events = {}".format(countevents)+"# in range = {}".format(countAngle))
        
        #Saving ctau to file 
        
        with open(outputfolder+"/ctau_plus.dat", 'a+') as fout:
                        #fout.write('# d0 [mm]' + '\n')
                        np.savetxt(fout, ctau_plus, fmt='%7f')   

In [43]:
# lifetime of chi_+
def lifetime_lhe(folder):
    
    outputfolder=folder+'_output'
    
    #reading pythia output
    files = sorted(glob.glob( folder+'/*.lhe'))
    print(files)

    os.makedirs(os.path.dirname(outputfolder+"/blank.dat"), exist_ok=True)
    
    #open lhe file
    for filenum, file in enumerate(files):
        
        param_point=re.search('\/(.+?).lhe', file).group(1)
        
        open(outputfolder+"/"+param_point+"_ctau_plus.dat","w")
        open(outputfolder+"/"+param_point+"_DM.dat","w")
        open(outputfolder+"/"+param_point+"_beta_plus.dat","w")
        open(outputfolder+"/"+param_point+"_gamma_plus.dat","w")
        open(outputfolder+"/"+param_point+"_betagamma_plus.dat","w")
        open(outputfolder+"/"+param_point+"_pT_plus.dat","w")
        
        lhe=[]
        ctau_plus=[]
        beta_plus=[]
        gamma_plus=[]
        betagamma_plus=[]
        pT_plus=[]
        DM=[]

        print(file)
        with open (file, 'r') as f:
            for rownum,row in enumerate(csv.reader(f)):
                if (len(row)>1):
                    lhe.append(row)

        print(len(lhe)) 

        countevents=0
        countV0=0
        for rownum ,row in enumerate(lhe):
            #print(lhe[rownum][0])
            #finding vertex
            print(row)
            if (row[0]=='<event>'):
                countevents+=1
            
            #calculating beta and gamma factors of chi_plus
            if (row[0]=='9000007') or (row[0]=='-9000007'):
                
                plus=np.array(row[6:10], float)
                print(plus)
                
                beta=np.sqrt(plus[0]**2+plus[1]**2+plus[2]**2)/plus[3]
                beta_plus.append(beta)
                gamma_plus.append(1/np.sqrt(1-beta**2))
                betagamma_plus.append(beta/np.sqrt(1-beta**2))
                pT_plus.append(np.sqrt(plus[0]**2+plus[1]**2))
            
                              
        #print(d0_l)
    
        #print("# events = {}".format(countevents)+"# e = {}".format(counte)+"# mu = {}".format(countmu)\
        #      +"# V(primary) = {}".format(countV0))
        
        #Saving to files 

                    
        with open(outputfolder+"/"+param_point+"_beta_plus.dat", 'a+') as fout:
                        np.savetxt(fout, beta_plus, fmt='%7f') 
                    
        with open(outputfolder+"/"+param_point+"_gamma_plus.dat", 'a+') as fout:
                        np.savetxt(fout, gamma_plus, fmt='%7f') 
                
        with open(outputfolder+"/"+param_point+"_betagamma_plus.dat", 'a+') as fout:
                        np.savetxt(fout, betagamma_plus, fmt='%7f') 
                
        with open(outputfolder+"/"+param_point+"_pT_plus.dat", 'a+') as fout:
                        np.savetxt(fout, pT_plus, fmt='%7f') 
                
        #print(DM)            
        #with open(outputfolder+"/DM.dat", 'a+') as fout:
        #                fout.write(DM)

In [44]:
#choosing var1 & var2 with respect to cut on var1

def chooseWithCut(marker,file1,file2,variable1,variable2,output_folder,cut_file1):
    var1 = np.array(genfromtxt(file1))
    var2 = np.array(genfromtxt(file2))
    
    print(var2[var1>cut_file1])
    
    var1New=var1[var1>cut_file1]
    var2New=var2[var1>cut_file1]
    
    oute=output_folder+"_cut/"
    
    #os.makedirs(os.path.dirname(oute), exist_ok=True)
    
    with open(output_folder+'/'+marker+variable1+'_cut.dat', 'w') as fout1:
        np.savetxt(fout1, var1New, fmt='%7f')
    
    with open(output_folder+'/'+marker+variable2+'_cut.dat', 'w') as fout2:
        np.savetxt(fout2, var2New, fmt='%7f')

In [45]:
def impact_with_cut(param_point, cut):
    
    folder='hepmc_output_'+param_point
    
    impact_d0(param_point)
    chooseWithCut(param_point,folder+'/pT_e.dat',folder+'/d0_e.dat','pT_e','d0_e',folder,cut)
    chooseWithCut(param_point,folder+'/pT_mu.dat',folder+'/d0_mu.dat','pT_mu','d0_mu',folder,cut)
    

In [59]:
lifetime_lhe('lhe_files_ctau2cm')

['lhe_files_ctau2cm/rr5.00e-08_mS2.75e+02_mT3.24e+02.lhe']
lhe_files_ctau2cm/rr5.00e-08_mS2.75e+02_mT3.24e+02.lhe
24
['generate p p > Z > psiplus psiminus ', ' psiplus > psil l+ vl ', ' (psiminus\\']
['p p > Z > psiplus psiminus ', ' psiplus > psil l+ vl ', ' (psiminus > psil l- vl~ )           #Process']
['#   To display more options', ' you can type the command:               *']
['# If you want to run Pythia', ' avoid more than 50k events in a run.    *']
['# lpp: 0=No PDF', ' 1=proton', ' -1=antiproton', ' 2=photon from proton', '      *']
['     230000    = lhaid     ! if pdlabel=lhapdf', ' this is the lhapdf number']
[' 0 = ickkw            ! 0 no matching', ' 1 MLM']
['# Generation bias', ' check the wiki page below for more information:   *']
[' None = bias_module  ! Bias type of bias', ' [None', ' ptj_bias', ' -custom_folder-]']
["# Minimum and maximum pt's (for max", ' -1 means no cut)                *']
[' {}\t= pt_max_pdg ! pt cut for other particles (syntax e.g. {6: 100', 

In [46]:
impact_with_cut('rr1.00e-08_mS2.75e+02_mT3.24e+02',20)

['hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa
3051106
# events = 5487# e = 6055# mu = 4013# V(primary) = 54
hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab
3051720
# events = 5438# e = 6095# mu = 3952# V(primary) = 52
hepmc_rr1.00e-08_mS2.75e+02_m

In [47]:
impact_with_cut('rr1.00e-08_mS2.85e+02_mT3.24e+02',20)

['hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa
3051933
# events = 5501# e = 5991# mu = 3941# V(primary) = 51
hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab
3052377
# events = 5440# e = 6023# mu = 3963# V(primary) = 45
hepmc_rr1.00e-08_mS2.85e+02_m

In [48]:
impact_with_cut('rr1.00e-08_mS2.95e+02_mT3.24e+02',20)

['hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa
3050801
# events = 5494# e = 6046# mu = 3796# V(primary) = 34
hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab
3051332
# events = 5491# e = 5997# mu = 3903# V(primary) = 48
hepmc_rr1.00e-08_mS2.95e+02_m

In [49]:
impact_with_cut('rr1.00e-08_mS3.04e+02_mT3.24e+02',20)

['hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa
3053267
# events = 5410# e = 5998# mu = 3701# V(primary) = 37
hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab
3050235
# events = 5523# e = 5943# mu = 3906# V(primary) = 35
hepmc_rr1.00e-08_mS3.04e+02_m

In [50]:
impact_with_cut('rr1.00e-08_mS3.14e+02_mT3.24e+02',20)

['hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa
3051609
# events = 5499# e = 5814# mu = 3556# V(primary) = 55
hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab
3054176
# events = 5455# e = 5810# mu = 3612# V(primary) = 50
hepmc_rr1.00e-08_mS3.14e+02_m

In [51]:
impact_with_cut('rr1.00e-08_mS1.00e+02_mT1.20e+02',20)

['hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcah']
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcaa
3028494
# events = 6285# e = 6494# mu = 4400# V(primary) = 45
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcab
3030835
# events = 6188# e = 6589# mu = 4132# V(primary) = 49
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcac
3029244
# events = 6256# e = 6503# mu = 4354# V(primary) = 56
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/ta

In [52]:
impact_with_cut('rr1.00e-08_mS2.00e+02_mT2.20e+02',20)

['hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcai']
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcaa
3044729
# events = 5788# e = 6247# mu = 4013# V(primary) = 44
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcab
3045156
# events = 5745# e = 6202# mu = 3960# V(primary) = 40
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcac
3045441
# events = 5696# e = 610

In [53]:
impact_with_cut('rr1.00e-08_mS4.00e+02_mT4.20e+02',20)

['hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcaa
3055698
# events = 5278# e = 5776# mu = 3698# V(primary) = 66
hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcab
3057696
# events = 5266# e = 5763# mu = 3614# V(primary) = 43
hepmc_rr1.00e-08_mS4.00e+02_m

In [54]:
impact_with_cut('rr1.00e-08_mS5.00e+02_mT5.20e+02',20)

['hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcaa
3060086
# events = 5135# e = 5779# mu = 3520# V(primary) = 37
hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcab
3060542
# events = 5098# e = 5587# mu = 3557# V(primary) = 46
hepmc_rr1.00e-08_mS5.00e+02_m

In [14]:
lifetime('rr1.00e-08_mS2.75e+02_mT3.24e+02')

['hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa
3051106
hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab
3051720
hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac
3052099
hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_4_pythia8_events.h

In [15]:
lifetime('rr1.00e-08_mS2.85e+02_mT3.24e+02')

['hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa
3051933
hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab
3052377
hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac
3051675
hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_4_pythia8_events.h

In [16]:
lifetime('rr1.00e-08_mS2.95e+02_mT3.24e+02')

['hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa
3050801
hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab
3051332
hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac
3053360
hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_4_pythia8_events.h

In [17]:
lifetime('rr1.00e-08_mS3.04e+02_mT3.24e+02')

['hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa
3053267
hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab
3050235
hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac
3052466
hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_4_pythia8_events.h

In [18]:
lifetime('rr1.00e-08_mS3.14e+02_mT3.24e+02')

['hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcaa
3051609
hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcab
3054176
hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.hepmcac
3052754
hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_4_pythia8_events.h

In [55]:
lifetime('rr1.00e-08_mS1.00e+02_mT1.20e+02')

['hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcah']
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcaa
3028494
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcab
3030835
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcac
3029244
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcad
3030308
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia8_events.hepmcae
3030445
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_4_pythia

In [56]:
lifetime('rr1.00e-08_mS2.00e+02_mT2.20e+02')

['hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcai']
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcaa
3044729
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcab
3045156
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcac
3045441
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_events.hepmcad
3043865
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_4_pythia8_eve

In [57]:
lifetime('rr1.00e-08_mS4.00e+02_mT4.20e+02')

['hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcaa
3055698
hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcab
3057696
hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.hepmcac
3057655
hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_4_pythia8_events.h

In [58]:
lifetime('rr1.00e-08_mS5.00e+02_mT5.20e+02')

['hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcaa
3060086
hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcab
3060542
hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.hepmcac
3061112
hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_4_pythia8_events.h

In [71]:
impact_with_cut('rr5.00e-08_mS3.04e+02_mT3.24e+02',20)

['hepmc_rr5.00e-08_mS3.04e+02_mT3.24e+02/pythia.hepmcaa', 'hepmc_rr5.00e-08_mS3.04e+02_mT3.24e+02/pythia.hepmcab', 'hepmc_rr5.00e-08_mS3.04e+02_mT3.24e+02/pythia.hepmcac', 'hepmc_rr5.00e-08_mS3.04e+02_mT3.24e+02/pythia.hepmcad', 'hepmc_rr5.00e-08_mS3.04e+02_mT3.24e+02/pythia.hepmcae', 'hepmc_rr5.00e-08_mS3.04e+02_mT3.24e+02/pythia.hepmcaf', 'hepmc_rr5.00e-08_mS3.04e+02_mT3.24e+02/pythia.hepmcag', 'hepmc_rr5.00e-08_mS3.04e+02_mT3.24e+02/pythia.hepmcah', 'hepmc_rr5.00e-08_mS3.04e+02_mT3.24e+02/pythia.hepmcai', 'hepmc_rr5.00e-08_mS3.04e+02_mT3.24e+02/pythia.hepmcaj']
hepmc_rr5.00e-08_mS3.04e+02_mT3.24e+02/pythia.hepmcaa
3052440
# events = 5451# e = 5759# mu = 3887# V(primary) = 60
hepmc_rr5.00e-08_mS3.04e+02_mT3.24e+02/pythia.hepmcab
3052511
# events = 5437# e = 5918# mu = 3799# V(primary) = 63
hepmc_rr5.00e-08_mS3.04e+02_mT3.24e+02/pythia.hepmcac
3053554
# events = 5455# e = 5908# mu = 3705# V(primary) = 44
hepmc_rr5.00e-08_mS3.04e+02_mT3.24e+02/pythia.hepmcad
3051673
# events = 5479# e 

In [72]:
lifetime('rr5.00e-08_mS3.05e+02_mT3.24e+02')

hepmc_rr5.00e-08_mS3.05e+02_mT3.24e+02
[]


In [27]:
# devinding lifetime in 3 peaces

var1 = np.array(genfromtxt('hepmc_output/ctau_plus.dat'))


low=var1[var1<0.1]
middle=var1[(0.1<var1)&(var1<10)]
high=var1[10<var1]


out="histos/lifetime_split"

os.makedirs(os.path.dirname(out), exist_ok=True)

with open(out+'/ctau_low.dat', 'w') as fout:
    np.savetxt(fout, low, fmt='%7f')
    
with open(out+'/ctau_middle.dat', 'w') as fout:
    np.savetxt(fout, middle, fmt='%7f')
    
with open(out+'/ctau_high.dat', 'w') as fout:
    np.savetxt(fout, high, fmt='%7f')